In [7]:
from strands import Agent, tool, ToolContext
from strands_tools import calculator
from strands.models import BedrockModel

bedrock_model = BedrockModel(
    model_id="apac.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="ap-northeast-2",
    temperature=0.3
)

In [8]:
# Tool with invocation_state
@tool(context=True)
def save_research(data: str, tool_context: ToolContext) -> str:
    session_id = tool_context.invocation_state.get("session_id")
    print(f"Saving for session: {session_id}")
    return f"Saved: {data}"

In [9]:
# Create specialized agents as tools with invocation_state access
@tool(context=True)
def research_assistant(query: str, tool_context: ToolContext) -> str:
    """Research specialist that gathers information and saves it."""
    session_id = tool_context.invocation_state.get("session_id")
    agent = Agent(
        name="researcher",
        model=bedrock_model,
        system_prompt=f"You are a research specialist for session {session_id}. Gather comprehensive information.",
        tools=[save_research]
    )
    response = agent(query, invocation_state=tool_context.invocation_state)
    return str(response)

@tool(context=True)
def analysis_assistant(query: str, tool_context: ToolContext) -> str:
    """Analysis specialist that processes research data."""
    session_id = tool_context.invocation_state.get("session_id")
    agent = Agent(
        name="analyzer",
        model=bedrock_model,
        system_prompt=f"You are an analysis specialist for session {session_id}. Analyze data and provide insights.",
        tools=[calculator]
    )
    response = agent(query, invocation_state=tool_context.invocation_state)
    return str(response)

@tool(context=True)
def writing_assistant(query: str, tool_context: ToolContext) -> str:
    """Writing specialist that creates final reports."""
    session_id = tool_context.invocation_state.get("session_id")
    agent = Agent(
        name="writer",
        model=bedrock_model,
        system_prompt=f"You are a writing specialist for session {session_id}. Create comprehensive reports.",
        tools=[]
    )
    response = agent(query, invocation_state=tool_context.invocation_state)
    return str(response)

In [10]:
# Create orchestrator agent with specialized agents as tools
orchestrator = Agent(
    name="orchestrator",
    model=bedrock_model,
    system_prompt="""You are an orchestrator that coordinates specialized agents:
    - Use research_assistant for gathering information
    - Use analysis_assistant for analyzing data
    - Use writing_assistant for creating final reports
    
    Call them in sequence: research → analysis → writing""",
    tools=[research_assistant, analysis_assistant, writing_assistant]
)

In [11]:
# Run orchestrator with invocation_state
result = orchestrator(
    "Research AWS Lambda pricing and create a summary report",
    invocation_state={"session_id": "session-123"}
)
print(result)

I'll help you research AWS Lambda pricing and create a comprehensive summary report. Let me coordinate with my specialized agents to gather information, analyze it, and create a final report.
Tool #1: research_assistant
I'll gather comprehensive information about AWS Lambda pricing for you.
Tool #1: save_research
Saving for session: session-123
I've compiled comprehensive research on AWS Lambda pricing covering all the aspects you requested. Here's a summary of the key findings:

## Key Highlights:

**Pricing Model**: Pay-per-use with two main components:
- **Requests**: $0.20 per 1 million requests
- **Duration**: $0.0000166667 per GB-second (x86) or $0.0000133334 per GB-second (ARM/Graviton2)

**Free Tier**: Very generous with 1 million free requests and 400,000 GB-seconds monthly (permanent for all customers)

**Regional Variations**: US regions are cheapest, with premiums of 10-50% in other regions

**Recent Updates**: 
- Graviton2 ARM processors offer 20% cost savings
- Container 

╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 0.20 / 1000000      │                                                                            │
│  │ Result    │ 2.0000000000e-07    │                                                                            │
│  │ Format    │ Scientific notation │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Tool #2: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬────────────────────────────────────────────────────╮                                             │
│  │ Operation │ Evaluate Expression                                │                                             │
│  │ Input     │ (0.0000166667 - 0.0000133334) / 0.0000166667 * 100 │                                             │
│  │ Result    │ 20                                                 │                                             │
│  ╰───────────┴────────────────────────────────────────────────────╯                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Tool #3: calculator


...


Tool #15: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬────────────────────────────╮                                                                     │
│  │ Operation │ Evaluate Expression        │                                                                     │
│  │ Input     │ (1710 - 1370) / 1710 * 100 │                                                                     │
│  │ Result    │ 20                         │                                                                     │
│  ╰───────────┴────────────────────────────╯                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Now let me analyze regional premium impact:
Tool #16: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 1710 * 1.5          │                                                                            │
│  │ Result    │ 2564                │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# AWS Lambda Pricing Analysis: Cost Drivers, Optimization Strategies & Scenarios

## **Key Cost Drivers Analysis**

### 1. **Primary Cost Components**
- **Request Charges**: $0.0000002 per request ($0.20 per 1M requests)
- **Compute Duration**: 
  - x86: $0.0000166667 per GB-second
  - ARM (Graviton2): $0.0000133334 per GB-second (20% savings)
- **Regional Variations**: 10-50% premium in certain regions

### 2. **Cost Driver Hierarchy**
1. **Memory allocation × Execution time** (typically 70-90% of costs for compute-intensive workloads)
2. **Request frequency** (dominant for high-volume, short-duration functions)
3. **Regional deployment** (can add 10-50% premium)
4. **Architecture choice** (ARM can reduce costs by 20%)

### 3. **Free Tier Impact**
- **Monthly allowances**: 1M requests + 400K GB-seconds
- **Equivalent to**: ~0.154 GB continuously running for a month
- **Break-even point**: Most production workloads exceed free tier quickly

## **Optimization Strategies**

### 1. **Memo